<a href="https://colab.research.google.com/github/HUMINTING/callte-detetion-detail/blob/main/yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Google ドライブを Google Colab 仮想マシンのファイルシステムにマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

検証セットデータをcolabにアップロードする

In [ ]:
import shutil
import os

source_dir = "/content/drive/MyDrive/new_estrus/yolo_7000/val_txt"
target_dir = "/content/dataset/val"

for filename in os.listdir(source_dir):
    shutil.copy(os.path.join(source_dir, filename), target_dir)


学習セットデータのラベルをcolabにアップロードする

In [ ]:
import shutil
import os

source_dir = "/content/drive/MyDrive/new_estrus/label/labels"
target_dir = "/content/dataset/train/labels"

for filename in os.listdir(source_dir):
    shutil.copy(os.path.join(source_dir, filename), target_dir)


学習セットデータの画像をcolabにアップロードする

In [ ]:
import shutil
import os

source_dir = "/content/drive/MyDrive/new_estrus/label/images"
target_dir = "/content/dataset/train/images"

for filename in os.listdir(source_dir):
    shutil.copy(os.path.join(source_dir, filename), target_dir)


モデルをcolabにアップロードする

In [ ]:
!cp -r "/content/drive/MyDrive/pretrained_model/yolov8-main" "/content"


In [ ]:
!pip install thop

yolov8_main ファイルを開きます

In [ ]:
%cd /content/yolov8-main


In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install timm


In [ ]:
!pip install einops


In [ ]:
!pip install -r /content/yolov8-main/requirements.txt


In [ ]:
!pip install efficientnet_pytorch


In [ ]:
!pip install Pillow


学習する

In [ ]:
!python train.py

検証する（実際にData-yolov8-improveテストセットを使用してこのステップを実行します）

In [ ]:
!python val.py

結果をGoogle ドライブに保存する

In [ ]:
import shutil

source_dir = "/content/result"
dest_dir = "/content/drive/MyDrive/new_estrus/train_results_focal"

shutil.copytree(source_dir, dest_dir)


分類テストする(実際にData-YOLOv8vsRetinanetテストセットを使用してこのステップを実行します）

In [ ]:
import torch
from ultralytics import YOLO
import os
import json
import gc

# 设置置信度阈值和批处理大小
confidence_threshold = 0.5

# 加载模型
model = YOLO('/content/drive/MyDrive/new_estrus/train_results_focal/weights/best.pt')

# 数据集路径
val_data_path = "/content/drive/MyDrive/new_estrus/test/true"

# 初始化统计数据
statistics = {
    'TP_estrus': 0, 'TN_estrus': 0, 'FP_estrus': 0, 'FN_estrus': 0,
    'TP_standing_mounting': 0, 'TN_standing_mounting': 0, 'FP_standing_mounting': 0, 'FN_standing_mounting': 0,
    'TP_mounting': 0, 'TN_mounting': 0, 'FP_mounting': 0, 'FN_mounting': 0
}

# 收集所有图像文件的路径
image_paths = [os.path.join(val_data_path, f) for f in os.listdir(val_data_path) if f.endswith(('.jpg', '.png'))]

# 处理图像文件
for image_path in image_paths:
    results = model.predict(image_path)

    for result in results:
        predicted_classes = [int(box.cls) for box in result.boxes if box.conf >= confidence_threshold] if result.boxes else []

        # 假设所有merged_true图片的真实标签都是1
        true_label = 1

        # 更新统计数据
        prediction_S_M = 1 if 0 in predicted_classes else 0
        prediction_M_Only = 1 if 1 in predicted_classes else 0
        prediction_AND = 1 if (0 in predicted_classes) and (1 in predicted_classes) else 0

        statistics['TP_standing_mounting'] += int(prediction_S_M == 1 and true_label == 1)
        statistics['TN_standing_mounting'] += int(prediction_S_M == 0 and true_label == 0)
        statistics['FP_standing_mounting'] += int(prediction_S_M == 1 and true_label == 0)
        statistics['FN_standing_mounting'] += int(prediction_S_M == 0 and true_label == 1)

        statistics['TP_mounting'] += int(prediction_M_Only == 1 and true_label == 1)
        statistics['TN_mounting'] += int(prediction_M_Only == 0 and true_label == 0)
        statistics['FP_mounting'] += int(prediction_M_Only == 1 and true_label == 0)
        statistics['FN_mounting'] += int(prediction_M_Only == 0 and true_label == 1)

        statistics['TP_estrus'] += int(prediction_AND == 1 and true_label == 1)
        statistics['TN_estrus'] += int(prediction_AND == 0 and true_label == 0)
        statistics['FP_estrus'] += int(prediction_AND == 1 and true_label == 0)
        statistics['FN_estrus'] += int(prediction_AND == 0 and true_label == 1)

    gc.collect()

# 计算精确度和召回率
def calculate_metrics(TP, TN, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return precision, recall

precision_standing_mounting, recall_standing_mounting = calculate_metrics(statistics['TP_standing_mounting'], statistics['TN_standing_mounting'], statistics['FP_standing_mounting'], statistics['FN_standing_mounting'])
precision_mounting, recall_mounting = calculate_metrics(statistics['TP_mounting'], statistics['TN_mounting'], statistics['FP_mounting'], statistics['FN_mounting'])
precision_estrus, recall_estrus = calculate_metrics(statistics['TP_estrus'], statistics['TN_estrus'], statistics['FP_estrus'], statistics['FN_estrus'])

# 输出结果
print(f"Standing Mounting - Precision: {precision_standing_mounting:.2f}, Recall: {recall_standing_mounting:.2f}")
print(f"Mounting - Precision: {precision_mounting:.2f}, Recall: {recall_mounting:.2f}")
print(f"Estrus - Precision: {precision_estrus:.2f}, Recall: {recall_estrus:.2f}")

# 保存统计结果的路径
statistics_save_path = '/content/drive/MyDrive/new_estrus/test/statistics_T.json'
# Save the statistics to a JSON file
with open(statistics_save_path, 'w') as file:
    json.dump(statistics, file, indent=4)

print(f"Statistics saved to {statistics_save_path}")



In [ ]:
import torch
from ultralytics import YOLO
import os
import json
import gc

# 设置置信度阈值和批处理大小
confidence_threshold = 0.5

# 加载模型
model = YOLO('/content/drive/MyDrive/new_estrus/train_results_focal/weights/best.pt')

# 数据集路径
val_data_path = "/content/drive/MyDrive/new_estrus/test/false"

# 初始化统计数据
statistics = {
    'TP_estrus': 0, 'TN_estrus': 0, 'FP_estrus': 0, 'FN_estrus': 0,
    'TP_standing_mounting': 0, 'TN_standing_mounting': 0, 'FP_standing_mounting': 0, 'FN_standing_mounting': 0,
    'TP_mounting': 0, 'TN_mounting': 0, 'FP_mounting': 0, 'FN_mounting': 0
}

# 收集所有图像文件的路径
image_paths = [os.path.join(val_data_path, f) for f in os.listdir(val_data_path) if f.endswith(('.jpg', '.png'))]

# 处理图像文件
for image_path in image_paths:
    results = model.predict(image_path)

    for result in results:
        predicted_classes = [int(box.cls) for box in result.boxes if box.conf >= confidence_threshold] if result.boxes else []

        # 假设所有merged_true图片的真实标签都是1
        true_label = 1

        # 更新统计数据
        prediction_S_M = 1 if 0 in predicted_classes else 0
        prediction_M_Only = 1 if 1 in predicted_classes else 0
        prediction_AND = 1 if (0 in predicted_classes) and (1 in predicted_classes) else 0

        statistics['TP_standing_mounting'] += int(prediction_S_M == 1 and true_label == 1)
        statistics['TN_standing_mounting'] += int(prediction_S_M == 0 and true_label == 0)
        statistics['FP_standing_mounting'] += int(prediction_S_M == 1 and true_label == 0)
        statistics['FN_standing_mounting'] += int(prediction_S_M == 0 and true_label == 1)

        statistics['TP_mounting'] += int(prediction_M_Only == 1 and true_label == 1)
        statistics['TN_mounting'] += int(prediction_M_Only == 0 and true_label == 0)
        statistics['FP_mounting'] += int(prediction_M_Only == 1 and true_label == 0)
        statistics['FN_mounting'] += int(prediction_M_Only == 0 and true_label == 1)

        statistics['TP_estrus'] += int(prediction_AND == 1 and true_label == 1)
        statistics['TN_estrus'] += int(prediction_AND == 0 and true_label == 0)
        statistics['FP_estrus'] += int(prediction_AND == 1 and true_label == 0)
        statistics['FN_estrus'] += int(prediction_AND == 0 and true_label == 1)

    gc.collect()

# 计算精确度和召回率
def calculate_metrics(TP, TN, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return precision, recall

precision_standing_mounting, recall_standing_mounting = calculate_metrics(statistics['TP_standing_mounting'], statistics['TN_standing_mounting'], statistics['FP_standing_mounting'], statistics['FN_standing_mounting'])
precision_mounting, recall_mounting = calculate_metrics(statistics['TP_mounting'], statistics['TN_mounting'], statistics['FP_mounting'], statistics['FN_mounting'])
precision_estrus, recall_estrus = calculate_metrics(statistics['TP_estrus'], statistics['TN_estrus'], statistics['FP_estrus'], statistics['FN_estrus'])

# 输出结果
print(f"Standing Mounting - Precision: {precision_standing_mounting:.2f}, Recall: {recall_standing_mounting:.2f}")
print(f"Mounting - Precision: {precision_mounting:.2f}, Recall: {recall_mounting:.2f}")
print(f"Estrus - Precision: {precision_estrus:.2f}, Recall: {recall_estrus:.2f}")

# 保存统计结果的路径
statistics_save_path = '/content/drive/MyDrive/new_estrus/test/statistics_F.json'
# Save the statistics to a JSON file
with open(statistics_save_path, 'w') as file:
    json.dump(statistics, file, indent=4)

print(f"Statistics saved to {statistics_save_path}")

